In [ ]:
!pip install transformers
!pip install pytorch-pretrained-bert

In [ ]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

In [ ]:
directory_path = 'data'

In [ ]:
hotel_df = pd.read_csv(os.path.join(directory_path,'hotel.csv'))
# jokes_df = pd.read_csv(os.path.join(directory_path,'jokes.csv'))
toxic_df = pd.read_csv(os.path.join(directory_path, 'jigsaw-toxic-comment-train.csv'))

In [ ]:
display(hotel_df.shape)
display(toxic_df.shape)

In [ ]:
hotel_df['REVIEW'] = hotel_df['reviews.text']
toxic_df['TOXICOMMENT'] = toxic_df['comment_text']

In [ ]:
hotel_df['REVIEW']

In [ ]:
toxic_df['TOXICOMMENT']

In [ ]:
hotel_df['REVIEW'] = hotel_df.REVIEW.str.lower()
toxic_df['TOXICOMMENT'] = toxic_df.TOXICOMMENT.str.lower()

In [ ]:
hotel_df['REVIEW'] = hotel_df['REVIEW'].map(lambda k: '[REVIEW] '+str(k))
toxic_df['TOXICOMMENT'] = toxic_df['TOXICOMMENT'].map(lambda k: '[TOXICOMMENT] '+str(k))

In [ ]:
changed_text_hotel=hotel_df.REVIEW.apply(lambda x:x+"\n"+"\n")
changed_text_toxic=toxic_df.TOXICOMMENT.apply(lambda x:x+"\n"+"\n")

In [ ]:
open(os.path.join(directory_path,'data_lm_hotel.txt'), "w").write(''.join(changed_text_hotel))

In [ ]:
open(os.path.join(directory_path,'data_lm_toxic.txt'), "w").write(''.join(changed_text_toxic))

In [ ]:
#%cd /content/drive/My Drive/Bert Training/pytorch-transformers/examples/lm_finetuning
import os
os.listdir(os.getcwd())

In [ ]:
!python3 lm_finetuning/pregenerate_training_data.py --train_corpus data/data_lm_hotel.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 2 --max_seq_len 128

In [ ]:
!python3 lm_finetuning/pregenerate_training_data.py --train_corpus data/data_lm_toxic.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 2 --max_seq_len 128

In [ ]:
os.listdir(os.getcwd())

In [ ]:
!python3 lm_finetuning/finetune_on_pregenerated.py --pregenerated_data training/ --bert_model bert-base-uncased --do_lower_case --train_batch_size 16  --output_dir finetuned_lm/ --epochs 2